In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

In [8]:
def review_parser_for_travel(input_review:str):
    model = ChatOpenAI(
        base_url="https://api.groq.com/openai/v1", 
        model="moonshotai/kimi-k2-instruct-0905",
        temperature=0.7
    )
    response_schemas = [
        ResponseSchema(name="destination", description="장소"),
        ResponseSchema(name="duration", description="숙박 일시"),
        ResponseSchema(name="budget", description="예산"),
        ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
        ResponseSchema(name="activities", description="destination에서 추천할 만한 활동 3가지를 리스트로 출력")
    ]

    # 파서 초기화
    parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = parser.get_format_instructions()

    template = """
    다음의 여행리뷰를 분석하시오: {review}

    {format_instructions}
    """

    prompt = ChatPromptTemplate.from_template(template)
    prompt = prompt.partial(format_instructions=format_instructions)

    review = input_review
    
    chain = prompt | model | parser
    return chain.invoke({"review": review})



In [6]:
review =  """
 지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

In [7]:
review_parser_for_travel(review)

{'destination': '부산',
 'duration': '2박 3일',
 'budget': '30만원',
 'rating': '4점',
 'activities': ['해운대 바다 구경', '자갈치시장에서 신선한 회 먹기', '감천문화마을 골목 산책']}